In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import transformers as tf
import warnings
warnings.filterwarnings('ignore')


C:\Users\Chris Mathew\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pandas as pd
# data from https://www.kaggle.com/datasets/balaka18/email-spam-classification-dataset-csv

data_sms=pd.read_csv("D:/tos_dataset/final_dataset.csv",encoding='utf8')

In [3]:
data_sms

,v2,v1
0,this service does not sell your personal data,0
1,this service will warn users about website mai...,0
2,you can request access and deletion of persona...,0
3,the user is informed about security practices,0
4,the service provides a complaint mechanism for...,0
...,...,...
1242,you agree that you will not: violate the profe...,0
1243,"for general inquiries, you may contact us onli...",0
1244,please do not post or add personal data to you...,0
1245,we collect personal data from you when you pro...,0


In [4]:
sample_data = data_sms[:1000]

In [5]:
ml_cls, tokenizer_cls, pt_weights = (tf.DistilBertModel, tf.DistilBertTokenizer, 'distilbert-base-uncased')

In [6]:

tzr = tokenizer_cls.from_pretrained(pt_weights)
mod = ml_cls.from_pretrained(pt_weights)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
from transformers import AutoTokenizer

# Assuming you want to use the 'bert-base-uncased' tokenizer
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

# Now you can use the 'tokenizer' object to tokenize your text



In [8]:
tokenized = sample_data['v2'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [9]:
## test the model accuracy on sample data
#sample_data = df['v2'].to_list()[4000:4100]
#sample_labels = df['label'].to_list()[4000:4100]

In [10]:
#token_sample_output = tokenizer(sample_data,padding=True,return_tensors="pt")
#output_prediction = model(**token_sample_output)
#numpy_output = output_prediction.logits.detach().numpy()
#import numpy as np
#preds = np.argmax(numpy_output, axis=-1)
#from sklearn.metrics import accuracy_score
#accuracy_score(sample_labels, preds)

In [11]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [12]:
np.array(padded).shape

(1000, 275)

In [13]:

attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(1000, 275)

In [14]:
from transformers import AutoModel, BertTokenizerFast
model=AutoModel.from_pretrained('distilbert-base-uncased')

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [15]:
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)
    


In [ ]:
features = last_hidden_states[0][:,0,:].numpy()

In [ ]:

labels = sample_data['v2']


In [ ]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf= RandomForestClassifier()
clf.fit(train_features, train_labels)

RandomForestClassifier()

In [ ]:
clf.score(test_features, test_labels)


0.392